# Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup and Imports

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [27]:
# For example: 
import os
from dotenv import load_dotenv

from typing import List, Any, Annotated
from pydantic import BaseModel, Field

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool
from lib.parsers import PydanticOutputParser, JsonOutputParser

from chromadb import PersistentClient
import chromadb


In [26]:
# Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
VOCAREUM_OPENAI_API_KEY=os.getenv("VOCAREUM_OPENAI_API_KEY")
VOCAREUM_BASE_URL=os.getenv("VOCAREUM_BASE_URL")

Setting up ChromaDB paths and names. Should be consistent with what you used in the `Udaplay_01.ipynb` file.

In [9]:
CHROMADB_PATH = "chromadb"
CHROMADB_COLLECTION_NAME = "udaplay"

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [ ]:
def retrieve_game(query: str, n_results: int=4) -> List:
    """
#    Semantic search: Finds most relevant results in the vector DB
#    args:
#    - query: a question about game industry. 
#    - n_results: number of results to return (default is 4)
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game
    """

    # collection.query(
        # query_texts=[query],
        # n_results=n_results,
        # include=['metadatas', 'documents', 'distances'] # TODO: look at collection.peek() to understand this better.
    # )

    # These have been created in Phase 1
    chroma_client = PersistentClient(path=CHROMADB_PATH)
    collection = chroma_client.get_collection(CHROMADB_COLLECTION_NAME)

    results = collection.query(
        query_texts=[query], 
        n_results=5
        )
    
    return results["documents"][0]

In [28]:
# To test the function
query = "Tell me about Gran Turismo"
# response = retrieve_game(query)
# print(response)

#### Evaluate Retrieval Tool

In [ ]:
class EvaluationReport(BaseModel):
    useful: bool = Field(..., description="Indicates if the retrieved documents are useful for answering the question")
    description: str = Field(..., description="Detailed explanation of the evaluation result, including why the documents are or are not useful")


def evaluate_retrieval(question: str, retrieved_docs: list) -> dict:
    """
    Based on the user's question and on the list of retrieved documents, this function 
    will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    Outputs:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """

    system_prompt = """
        You are an expert in evaluating the usability of documents for answering user queries.
        Your task is to evaluate if the provided sources are enough to respond the user query.
        Give a detailed explanation, so it's possible to take an action to accept it or not.
    """

    user_prompt = f"""
        User query:\n{question}\n
        Retrieved documents:\n{retrieved_docs}
    """

    messages = [
        SystemMessage(content=system_prompt),
        UserMessage(content=user_prompt)
    ]

    llm_judge = LLM(
        model="gpt-4.1-nano",
        temperature=0.0,
        api_key=VOCAREUM_OPENAI_API_KEY,
        base_url=VOCAREUM_BASE_URL
    )

    ai_message = llm_judge.invoke(input=messages, response_format=EvaluationReport)
    parser = JsonOutputParser()
    return parser.parse(ai_message)


# Use EvaluationReport to parse the result
# Tool Docstring:

In [ ]:
# To test the function
response_valid = evaluate_retrieval(query, response)
validated_response = EvaluationReport(**response_valid) # To validate format adheres to EvaluationReport
print(f"""
    Is the result useful? 
    {validated_response.useful}

    Detailed explanation:
    {validated_response.description}
""")


    Is the result useful? 
    True

    Detailed explanation:
    The retrieved documents include detailed information about the Gran Turismo series, including its different versions such as Gran Turismo 5 for PlayStation 3 and the original Gran Turismo for PlayStation 1. These descriptions highlight that Gran Turismo is a racing simulator with realistic physics, a wide selection of cars and tracks, and a significant impact on the racing game genre. Although some documents mention other games like Grand Theft Auto and Mario Kart, the relevant entries about Gran Turismo provide sufficient context to answer user queries about the game, its features, and its history.



#### Game Web Search Tool

In [ ]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 

### Agent

In [ ]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

In [ ]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes